In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display,clear_output
import pandas as pd
import matplotlib.pyplot as plt
import datajoint as dj
from pipeline import pipeline_tools, lab, experiment, behavior_foraging
import numpy as np
dj.conn()
#%matplotlib widgetsnbextension

Connecting rozmar@mesoscale-activity.datajoint.io:3306


DataJoint connection (connected) rozmar@mesoscale-activity.datajoint.io:3306

In [2]:
def merge_dataframes_with_nans(df_1,df_2,basiscol):
    basiscol = 'trial'
    colstoadd = list()
# =============================================================================
#     df_1 = df_behaviortrial
#     df_2 = df_reactiontimes
# =============================================================================
    for colnow in df_2.keys():
        if colnow not in df_1.keys():
            df_1[colnow] = np.nan
            colstoadd.append(colnow)
    for line in df_2.iterrows():
        for colname in colstoadd:
            df_1.loc[df_1[basiscol]==line[1][basiscol],colname]=line[1][colname]
    return df_1


In [3]:
wr_names = (experiment.Session()*lab.WaterRestriction()).fetch('water_restriction_number')
wr_names = np.unique(wr_names)


fig=plt.figure()
ax1=fig.add_axes([0,0,2,.8])
plt.close()
## functions related to widgets
def update_params(change=None):
    
    wr_name = wr_name_selector.value
    subject_id = (lab.WaterRestriction()&'water_restriction_number = "{}"'.format(wr_name)).fetch1('subject_id')
    df_sessions = pd.DataFrame(experiment.Session()&'subject_id = {}'.format(subject_id))
    session_selector.min = df_sessions['session'].min()
    session_selector.max = df_sessions['session'].max()
    
def plot_trials(dummy):
    ax1.clear()
    plottype = '2lickport'
    choice_filter = np.ones(20)/20
    local_filter = np.ones(10)/10

    with output:
        clear_output(wait=True)
        print('waiting for plot')
    wr_name = wr_name_selector.value
    subject_id = (lab.WaterRestriction()&'water_restriction_number = "{}"'.format(wr_name)).fetch1('subject_id')
    sessions = session_selector.value
    df_behaviortrial = pd.DataFrame(np.asarray((experiment.BehaviorTrial()* experiment.SessionTrial() * experiment.SessionBlock()* behavior_foraging.TrialReactionTime &
                                    'subject_id = {}'.format(subject_id) &
                                    'session >= {}'.format(sessions[0]) &
                                    'session <= {}'.format(sessions[1])).fetch('session',
                                                                               'trial',
                                                                               'p_reward_right',
                                                                               'p_reward_left',
                                                                               'p_reward_middle',
                                                                               'trial_choice',
                                                                               'outcome'
                                                                              )).T,columns = ['session',
                                                                                              'trial',
                                                                                              'p_reward_right',
                                                                                              'p_reward_left',
                                                                                              'p_reward_middle',
                                                                                              'trial_choice',
                                                                                              'outcome'])
    
    

    unique_sessions = df_behaviortrial['session'].unique()
    for session in unique_sessions:
        total_trials_so_far = (behavior_foraging.SessionStats()&'subject_id = {}'.format(subject_id) &'session < {}'.format(session)).fetch('session_total_trial_num')
        bias_check_trials_now = (behavior_foraging.SessionStats()&'subject_id = {}'.format(subject_id) &'session = {}'.format(session)).fetch1('session_bias_check_trial_num')
        total_trials_so_far =sum(total_trials_so_far)

        df_behaviortrial.loc[df_behaviortrial['session']==session, 'trial'] += total_trials_so_far
    
    if not bias_check_selector.value:
        realtraining = (df_behaviortrial['p_reward_left']<1) & (df_behaviortrial['p_reward_right']<1) & ((df_behaviortrial['p_reward_middle']<1) | df_behaviortrial['p_reward_middle'].isnull())
        df_behaviortrial = df_behaviortrial[realtraining]
        df_behaviortrial = df_behaviortrial.reset_index(drop=True)
    
    
    df_behaviortrial['trial_choice_plot'] = np.nan
    df_behaviortrial.loc[df_behaviortrial['trial_choice'] == 'left', 'trial_choice_plot'] = 0
    df_behaviortrial.loc[df_behaviortrial['trial_choice'] == 'right', 'trial_choice_plot'] = 1
    df_behaviortrial.loc[df_behaviortrial['trial_choice'] == 'middle', 'trial_choice_plot'] = .5

    trial_choice_plot_interpolated = df_behaviortrial['trial_choice_plot'].values
    nans, x= np.isnan(trial_choice_plot_interpolated), lambda z: z.nonzero()[0]
    trial_choice_plot_interpolated[nans]= np.interp(x(nans), x(~nans), trial_choice_plot_interpolated[~nans])

    if plottype == '2lickport':
        df_behaviortrial['reward_ratio']=df_behaviortrial['p_reward_right']/(df_behaviortrial['p_reward_right']+df_behaviortrial['p_reward_left'])
        bias = np.convolve(trial_choice_plot_interpolated,choice_filter,mode = 'valid')
        bias = np.concatenate((np.nan*np.ones(int(np.floor((len(choice_filter)-1)/2))),bias,np.nan*np.ones(int(np.ceil((len(choice_filter)-1)/2)))))
    elif plottype == '3lickport':
        df_behaviortrial['reward_ratio_1']=df_behaviortrial['p_reward_left']/(df_behaviortrial['p_reward_right']+df_behaviortrial['p_reward_left']+ df_behaviortrial['p_reward_middle'])
        df_behaviortrial['reward_ratio_2']=(df_behaviortrial['p_reward_left']+df_behaviortrial['p_reward_middle'])/(df_behaviortrial['p_reward_right']+df_behaviortrial['p_reward_left']+ df_behaviortrial['p_reward_middle'])
        #%
        leftchoices_filtered = np.convolve(df_behaviortrial['trial_choice'] == 'left',choice_filter,mode = 'valid')
        leftchoices_filtered = np.concatenate((np.nan*np.ones(int(np.floor((len(choice_filter)-1)/2))),leftchoices_filtered ,np.nan*np.ones(int(np.ceil((len(choice_filter)-1)/2)))))
        rightchoices_filtered = np.convolve(df_behaviortrial['trial_choice'] == 'right',choice_filter,mode = 'valid')
        rightchoices_filtered = np.concatenate((np.nan*np.ones(int(np.floor((len(choice_filter)-1)/2))),rightchoices_filtered ,np.nan*np.ones(int(np.ceil((len(choice_filter)-1)/2)))))
        middlechoices_filtered = np.convolve(df_behaviortrial['trial_choice'] == 'middle',choice_filter,mode = 'valid')
        middlechoices_filtered = np.concatenate((np.nan*np.ones(int(np.floor((len(choice_filter)-1)/2))),middlechoices_filtered ,np.nan*np.ones(int(np.ceil((len(choice_filter)-1)/2)))))
        allchoices_filtered = np.convolve(df_behaviortrial['trial_choice'] != 'none',choice_filter,mode = 'valid')
        allchoices_filtered = np.concatenate((np.nan*np.ones(int(np.floor((len(choice_filter)-1)/2))),allchoices_filtered ,np.nan*np.ones(int(np.ceil((len(choice_filter)-1)/2)))))

    rewarded = (df_behaviortrial['outcome']=='hit')
    unrewarded = (df_behaviortrial['outcome']=='miss')
    
    blockswitches = np.where(np.diff(df_behaviortrial['session'].values)>0)[0]
    if len(blockswitches)>0:
        for trialnum_now in blockswitches:
            ax1.plot([df_behaviortrial['trial'][trialnum_now],df_behaviortrial['trial'][trialnum_now]],[-.15,1.15],'b--')
            
    if plottype == '2lickport':
        if plot_each_choice.value:
            ax1.plot(df_behaviortrial['trial'][rewarded],df_behaviortrial['trial_choice_plot'][rewarded],'k|',color='black',markersize=30,markeredgewidth=2)
            ax1.plot(df_behaviortrial['trial'][unrewarded],df_behaviortrial['trial_choice_plot'][unrewarded],'|',color='gray',markersize=15,markeredgewidth=2)
        ax1.plot(df_behaviortrial['trial'],bias,'k-',label = 'choice')
        ax1.plot(df_behaviortrial['trial'],df_behaviortrial['reward_ratio'],'y-')
        ax1.set_yticks((0,1))
        ax1.set_yticklabels(('left','right'))
    elif plottype == '3lickport':
        ax1.stackplot(df_behaviortrial['trial'],  leftchoices_filtered/allchoices_filtered ,  middlechoices_filtered/allchoices_filtered ,  rightchoices_filtered/allchoices_filtered ,colors=['r','g','b'], alpha=0.3 )
        if plot_each_choice.value:
            ax1.plot(df_behaviortrial['trial'][rewarded],df_behaviortrial['trial_choice_plot'][rewarded],'k|',color='black',markersize=30,markeredgewidth=2)
            ax1.plot(df_behaviortrial['trial'][unrewarded],df_behaviortrial['trial_choice_plot'][unrewarded],'|',color='gray',markersize=15,markeredgewidth=2)
        ax1.plot(df_behaviortrial['trial'],df_behaviortrial['reward_ratio_1'],'y-')
        ax1.plot(df_behaviortrial['trial'],df_behaviortrial['reward_ratio_2'],'y-')
        ax1.set_yticks((0,.5,1))
        ax1.set_yticklabels(('left','middle','right'))
        ax1.set_ylim([-.1,1.1])
    #fig.canvas.draw_idle()
    with output:
        clear_output(wait=True)
        display(ax1.figure)



        
        
## widgets are defined here
border = widgets.Output(layout={'border': '1px solid black'})
output = widgets.Output()

task_selector = widgets.Dropdown(layout={'width': 'initial'},
                                    options=['2lickport','3lickport'],
                                    disabled=False)
task_selector.observe(update_params,'value')

wr_name_selector = widgets.Dropdown(layout={'width': 'initial'},
                                    options=wr_names,
                                    disabled=False)
wr_name_selector.observe(update_params,'value') # runs update params upon change in value


session_selector = widgets.IntRangeSlider(value=[5, 7],
                                          min=0,
                                          max=10,
                                          step=1,
                                          disabled=False,
                                          continuous_update=False,
                                          orientation='horizontal',
                                          readout=True,
                                          readout_format='d')

bias_check_selector = widgets.Checkbox(value=False,
                                       description='Include bias check',
                                       disabled=False,
                                       indent=False)

plot_button = widgets.Button(description="plot")
plot_button.on_click(plot_trials)

plot_each_choice = widgets.Checkbox(value=False,
                              description='Plot each choice',
                              disabled=False,
                              indent=False)

## layout of widgets
controls = widgets.HBox([widgets.VBox([widgets.Label('Task'), task_selector]),
                         widgets.VBox([widgets.Label('Subject'), wr_name_selector]),
                         widgets.VBox([widgets.Label('Sessions'),session_selector]),
                         widgets.VBox([bias_check_selector]),
                         plot_button])
plot_controls = widgets.HBox([plot_each_choice])
accordion = widgets.VBox(children=(controls,plot_controls), title=('Select data', 'Plot details'))
#accordion.set_title(0, 'Data')
#accordion.set_title(1, 'Plot')

display(accordion)
display(border)
display(output)

update_params()

Output(layout=Layout(border='1px solid black'))

Output()